In [1]:
import os, csv
import pandas as pd
from pyspark.sql import SparkSession, functions, types
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier, GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorAssembler, SQLTransformer
from pyspark.ml.regression import DecisionTreeRegressor, RandomForestRegressor
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

spark = SparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel('WARN')

# Read in data. We need to read in the first row of the CSV 
# to get the headers so we can dynamically create the schema.
DATA_DIR = '../../project_data/pipeline'
file_name = 'data_windows_05s.txt'

with open(os.path.join(DATA_DIR, file_name)) as f:
    reader = csv.reader(f, delimiter="\t")
    row1 = next(reader)

# Dynamically create schema. All cols except filename are numbers.
schema = types.StructType(
        [types.StructField(field_name, types.FloatType()) if field_name != 'filename' \
            else types.StructField(field_name, types.StringType()) for field_name in row1])

df = spark.read.csv(os.path.join(DATA_DIR, file_name), sep='\t', header=True, schema=schema)

In [3]:
df.columns

['Time',
 'trial_id',
 'label',
 'hea_Acc_X',
 'hea_Acc_Y',
 'hea_Acc_Z',
 'hea_Acc_res',
 'hea_Ang_Vel_X',
 'hea_Ang_Vel_Y',
 'hea_Ang_Vel_Z',
 'hea_Ang_Vel_res',
 'hea_Mag_Fie_X',
 'hea_Mag_Fie_Y',
 'hea_Mag_Fie_Z',
 'hea_Mag_Fie_res',
 'l_ank_Acc_X',
 'l_ank_Acc_Y',
 'l_ank_Acc_Z',
 'l_ank_Acc_res',
 'l_ank_Ang_Vel_X',
 'l_ank_Ang_Vel_Y',
 'l_ank_Ang_Vel_Z',
 'l_ank_Ang_Vel_res',
 'l_ank_Mag_Fie_X',
 'l_ank_Mag_Fie_Y',
 'l_ank_Mag_Fie_Z',
 'l_ank_Mag_Fie_res',
 'l_thi_Acc_X',
 'l_thi_Acc_Y',
 'l_thi_Acc_Z',
 'l_thi_Acc_res',
 'l_thi_Ang_Vel_X',
 'l_thi_Ang_Vel_Y',
 'l_thi_Ang_Vel_Z',
 'l_thi_Ang_Vel_res',
 'l_thi_Mag_Fie_X',
 'l_thi_Mag_Fie_Y',
 'l_thi_Mag_Fie_Z',
 'l_thi_Mag_Fie_res',
 'r_ank_Acc_X',
 'r_ank_Acc_Y',
 'r_ank_Acc_Z',
 'r_ank_Acc_res',
 'r_ank_Ang_Vel_X',
 'r_ank_Ang_Vel_Y',
 'r_ank_Ang_Vel_Z',
 'r_ank_Ang_Vel_res',
 'r_ank_Mag_Fie_X',
 'r_ank_Mag_Fie_Y',
 'r_ank_Mag_Fie_Z',
 'r_ank_Mag_Fie_res',
 'r_thi_Acc_X',
 'r_thi_Acc_Y',
 'r_thi_Acc_Z',
 'r_thi_Acc_res',
 'r_th

In [4]:
# confirm the labels are right
labels = df.select('label').collect()
falls = [l for l in labels if l[0] == 1]
non_falls = [l for l in labels if l[0] == 0]
print("Falls: {}, Non-falls: {}".format(len(falls), len(non_falls)))

# Get all columns except trial_id and label as features
# May need to tweak this if using non-windowed data
#max_cols = [col for col in df.columns if col.endswith('max')] + ['trial', 'label', 'start_time']
#df = df.select(max_cols)
features = [col for col in df.columns if col not in ['trial_id', 'label', 'Time']]
train, validation = df.randomSplit([0.75, 0.25])

Falls: 6300, Non-falls: 12290


In [20]:
## ALGORITHM 1: Random Forest Classifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Assemble Pipeline
assembler = VectorAssembler(
    inputCols=features,
    outputCol='features')
classifier = RandomForestClassifier(labelCol='label', maxDepth=20, numTrees=64)
pipeline = Pipeline(stages=[assembler, classifier])

# Set params to test
# maxDepth= [10, 20]
# numTrees =[32, 64]
# numFolds = 3
maxDepth= [20]
numTrees =[64]
numFolds = 3

paramGrid = ParamGridBuilder() \
    .addGrid(classifier.maxDepth, maxDepth) \
    .addGrid(classifier.numTrees, numTrees) \
    .build()

cv = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName='weightedPrecision'),
                          numFolds=numFolds)  
model = cv.fit(train)

In [21]:
best_pipeline = model.bestModel
print("Best maxDepth: {}".format(best_pipeline.stages[1].getOrDefault('maxDepth')))
print("Best numTrees: {}".format(best_pipeline.stages[1].getOrDefault('numTrees')))

predictions = model.transform(validation)

evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))
print("Test Area Under PR: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})))

evaluator = MulticlassClassificationEvaluator()
print("f1: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "f1"})))
print("Weighted Precision: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})))
print("Weighted Recall: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})))
print("Accuracy: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})))

Best maxDepth: 20
Best numTrees: 64
Test Area Under ROC: 0.9855275389709671
Test Area Under PR: 0.9753876946106923
f1: 0.9225502940258289
Weighted Precision: 0.9263599615250787
Weighted Recall: 0.9240506329113924
Accuracy: 0.9240506329113924


In [22]:
best_pipeline.write().overwrite().save('models/dt_0_5s_all_features')

In [23]:
#https://www.timlrx.com/2018/06/19/feature-selection-using-feature-importance-score-creating-a-pyspark-estimator/
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

f = ExtractFeatureImp(best_pipeline.stages[-1].featureImportances, predictions, "features")
top_20_features = list(f['name'].head(20).values)
f.head(20)

idx                 name     score
6     6  r_ank_Mag_Fie_X_max  0.050648
15   15  l_ank_Mag_Fie_X_max  0.046906
54   54        wai_Acc_X_max  0.040184
45   45        ste_Acc_X_max  0.038226
47   47        ste_Acc_Z_max  0.033926
18   18      r_thi_Acc_X_max  0.029278
24   24  r_thi_Mag_Fie_X_max  0.028458
36   36        hea_Acc_X_max  0.027305
43   43    hea_Mag_Fie_Y_max  0.023045
60   60    wai_Mag_Fie_X_max  0.022499
33   33  l_thi_Mag_Fie_X_max  0.022131
51   51    ste_Mag_Fie_X_max  0.022041
8     8  r_ank_Mag_Fie_Z_max  0.021199
17   17  l_ank_Mag_Fie_Z_max  0.019788
26   26  r_thi_Mag_Fie_Z_max  0.019388
27   27      l_thi_Acc_X_max  0.019360
55   55        wai_Acc_Y_max  0.019018
19   19      r_thi_Acc_Y_max  0.018704
0     0      r_ank_Acc_X_max  0.017971
38   38        hea_Acc_Z_max  0.017792

In [24]:
# Try again with just top 20 features
features = top_20_features

# Assemble Pipeline
assembler = VectorAssembler(
    inputCols=features,
    outputCol='features')
classifier = RandomForestClassifier(labelCol='label', maxDepth=20, numTrees=64)
pipeline = Pipeline(stages=[assembler, classifier])
model = pipeline.fit(train)
model.write().overwrite().save('models/dt_0_5s_top20_features')

In [36]:
# Make predictions
model_path = 'models/dt_0_5s_top20_features'
model = PipelineModel.load(model_path)
predictions = model.transform(validation)

In [37]:
# Access raw probability
from pyspark.sql.functions import udf

def access_vector(vector, index):
    try:
        return float(vector[index])
    except ValueError:
        return None

av_udf = udf(access_vector, types.DoubleType())
predictions = predictions.withColumn('raw_probability', av_udf("probability", functions.lit(1)))\
                .withColumnRenamed('prediction', 'prediction_old')
threshold = 0.55
predictions = predictions.withColumn('prediction', \
                functions.when(predictions['raw_probability'] > threshold, 1.0).otherwise(0.0))
predictions.select(['label', 'probability', 'raw_probability','prediction', 'prediction_old']).show()

+-----+--------------------+------------------+----------+--------------+
|label|         probability|   raw_probability|prediction|prediction_old|
+-----+--------------------+------------------+----------+--------------+
|  1.0|[0.14464649983714...|0.8553535001628565|       1.0|           1.0|
|  1.0|[0.29734629054367...|0.7026537094563275|       1.0|           1.0|
|  1.0| [0.015625,0.984375]|          0.984375|       1.0|           1.0|
|  1.0|           [0.0,1.0]|               1.0|       1.0|           1.0|
|  1.0|[0.06577892588090...|0.9342210741190912|       1.0|           1.0|
|  1.0|[0.05034313923433...| 0.949656860765662|       1.0|           1.0|
|  1.0|[0.13589561352651...|0.8641043864734872|       1.0|           1.0|
|  1.0|[0.21647594701877...|0.7835240529812219|       1.0|           1.0|
|  1.0| [0.078125,0.921875]|          0.921875|       1.0|           1.0|
|  1.0|[0.12827892588090...|0.8717210741190912|       1.0|           1.0|
|  1.0|           [0.0,1.0]|          

In [38]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))
print("Test Area Under PR: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})))

evaluator = MulticlassClassificationEvaluator()
print("f1: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "f1"})))
print("Weighted Precision: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})))
print("Weighted Recall: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})))
print("Accuracy: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})))

Test Area Under ROC: 0.9837321915894449
Test Area Under PR: 0.9724059533012722
f1: 0.9127703016851516
Weighted Precision: 0.9193670689022995
Weighted Recall: 0.915039691053422
Accuracy: 0.915039691053422


In [40]:
# ALGORITHM 2: Gradient Boosted Trees
# Can either train it here or load what has already been trained

# Assemble Pipeline
assembler = VectorAssembler(
    inputCols=features,
    outputCol='features')
classifier = GBTClassifier(maxIter=50, maxDepth=10, maxBins=50,seed=1)
pipeline = Pipeline(stages=[assembler, classifier])

# Set params to test
# maxDepth= [10, 20]
# maxBins =[32, 64]
# maxIter = [10, 20, 30]
maxDepth= [10]
maxBins =[64]
maxIter = [30]

paramGrid = ParamGridBuilder() \
    .addGrid(classifier.maxDepth, maxDepth) \
    .addGrid(classifier.maxBins, maxBins) \
    .addGrid(classifier.maxIter, maxIter) \
    .build()

cv = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName='weightedPrecision'),
                          numFolds=numFolds)  
model = cv.fit(train)

In [42]:
best_pipeline = model.bestModel
best_pipeline.write().overwrite().save('models/gbt_0_5s_all_features')

In [50]:
print("Best maxDepth: {}".format(best_pipeline.stages[1].getOrDefault('maxDepth')))
print("Best maxIter: {}".format(best_pipeline.stages[1].getOrDefault('maxIter')))
print("Best maxBins: {}".format(best_pipeline.stages[1].getOrDefault('maxBins')))

predictions = model.transform(validation)

evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))
print("Test Area Under PR: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})))

evaluator = MulticlassClassificationEvaluator()
print("f1: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "f1"})))
print("Weighted Precision: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})))
print("Weighted Recall: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})))
print("Accuracy: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})))

Best maxDepth: 10
Best maxIter: 30
Best maxBins: 64
Test Area Under ROC: 0.9704186192080355
Test Area Under PR: 0.9515248533937766
f1: 0.9075719700323568
Weighted Precision: 0.9079829306848832
Weighted Recall: 0.908388757777301
Accuracy: 0.908388757777301


In [44]:
predictions.select(['label', 'probability','prediction']).show()

+-----+--------------------+----------+
|label|         probability|prediction|
+-----+--------------------+----------+
|  1.0|[0.05447276757055...|       1.0|
|  1.0|[0.26492049469050...|       1.0|
|  1.0|[0.08294592127013...|       1.0|
|  1.0|[0.08294592127013...|       1.0|
|  1.0|[0.04586156863346...|       1.0|
|  1.0|[0.03073732239260...|       1.0|
|  1.0|[0.28626186917887...|       1.0|
|  1.0|[0.26491154780817...|       1.0|
|  1.0|[0.08225472650318...|       1.0|
|  1.0|[0.05051889318388...|       1.0|
|  1.0|[0.08294592127013...|       1.0|
|  1.0|[0.08166008037762...|       1.0|
|  1.0|[0.47851954074398...|       1.0|
|  1.0|[0.10578376565282...|       1.0|
|  1.0|[0.03618543924066...|       1.0|
|  1.0|[0.13892157690563...|       1.0|
|  1.0|[0.04805027991886...|       1.0|
|  1.0|[0.09638732895984...|       1.0|
|  1.0|[0.04669628492147...|       1.0|
|  1.0|[0.08803554739088...|       1.0|
+-----+--------------------+----------+
only showing top 20 rows



In [51]:
predictions = predictions.withColumn('raw_probability', av_udf("probability", functions.lit(1)))\
                .withColumnRenamed('prediction', 'prediction_old')
threshold = 0.6
predictions = predictions.withColumn('prediction', \
                functions.when(predictions['raw_probability'] > threshold, 1.0).otherwise(0.0))
predictions.select(['label', 'probability', 'raw_probability','prediction', 'prediction_old']).show()

+-----+--------------------+------------------+----------+--------------+
|label|         probability|   raw_probability|prediction|prediction_old|
+-----+--------------------+------------------+----------+--------------+
|  1.0|[0.05447276757055...|0.9455272324294439|       1.0|           1.0|
|  1.0|[0.26492049469050...|0.7350795053094934|       1.0|           1.0|
|  1.0|[0.08294592127013...|0.9170540787298627|       1.0|           1.0|
|  1.0|[0.08294592127013...|0.9170540787298627|       1.0|           1.0|
|  1.0|[0.04586156863346...|0.9541384313665309|       1.0|           1.0|
|  1.0|[0.03073732239260...|0.9692626776073929|       1.0|           1.0|
|  1.0|[0.28626186917887...|0.7137381308211279|       1.0|           1.0|
|  1.0|[0.26491154780817...|0.7350884521918211|       1.0|           1.0|
|  1.0|[0.08225472650318...|0.9177452734968162|       1.0|           1.0|
|  1.0|[0.05051889318388...|0.9494811068161193|       1.0|           1.0|
|  1.0|[0.08294592127013...|0.91705407

In [52]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))
print("Test Area Under PR: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})))

evaluator = MulticlassClassificationEvaluator()
print("f1: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "f1"})))
print("Weighted Precision: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})))
print("Weighted Recall: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})))
print("Accuracy: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})))

Test Area Under ROC: 0.9704186192080385
Test Area Under PR: 0.9515248533937767
f1: 0.9018524972855317
Weighted Precision: 0.9047834807890824
Weighted Recall: 0.9036687406136021
Accuracy: 0.9036687406136023


In [64]:
# ALGORITHM 3: MULTILAYER PERCEPTRON
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

layers = [len(features), 12, 6, 2]
classifier = MultilayerPerceptronClassifier(maxIter=10000, layers=layers, blockSize=128, seed=1234)
pipeline = Pipeline(stages=[assembler, classifier])
model = pipeline.fit(train)


In [65]:
predictions = model.transform(validation)

In [66]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))
print("Test Area Under PR: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})))

evaluator = MulticlassClassificationEvaluator()
print("f1: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "f1"})))
print("Weighted Precision: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})))
print("Weighted Recall: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})))
print("Accuracy: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})))

Test Area Under ROC: 0.8797668191498886
Test Area Under PR: 0.8161951516675431
f1: 0.7625227135087147
Weighted Precision: 0.8273740237511535
Weighted Recall: 0.7895301437459773
Accuracy: 0.7895301437459773


In [63]:
print(len(features))

20
